In [6]:
import torch, sys
sys.path.append('/home/dgotzens/scripts')
import pdfdefaults as pdf
import matplotlib.pyplot as plt
from matplotlib.ticker import EngFormatter

N = 2**16
M = 512
T = 1000
pi = 3.141592653589 

fs = 22e6
Ts = 1/fs

f0 = 5e6
omega0 = 2*pi*f0
f = torch.linspace(4e6,7e6,T)[:,None]
omega = 2*pi*f
m = torch.arange(M)[None,:]

yid = 1*torch.exp(1j*omega*m*Ts)
y = yid + 0.9*torch.exp(1j*omega0*m*Ts) + 0.01*torch.randn((T,M), dtype=torch.cfloat)
# y = 0.1*torch.exp(1j*omega0*m*Ts)

print(yid.shape, y.shape)

torch.Size([1000, 512]) torch.Size([1000, 512])


In [7]:
w = torch.hann_window(M)[None,:]
Yid = torch.fft.fft(w*yid,n=N,dim=-1)
Y = torch.fft.fft(w*y,n=N,dim=-1)


In [20]:
pdf.setup()
dBr = lambda x : 20*(x.abs()/x.abs().max()).log10()
fig,ax = plt.subplots(2,2, sharex=True, sharey=True, layout='constrained')
fig.set_size_inches(pdf.a4_textwidth, 0.6*pdf.a4_textwidth)

ax[0,0].imshow(dBr(Yid[:,:N//2].T), origin='lower', aspect=T/N)
ax[0,0].set_title(r'$20log|Y_{id}|$')
ax[0,1].imshow(dBr(Y[:,:N//2].T), origin='lower', aspect=T/N)
ax[0,1].set_title(r'$20log|Y_{}|$')
ax[1,1].imshow(Y[:,:N//2].T.angle(), origin='lower', aspect=T/N)
ax[1,1].set_title(r'arg$Y_{}$')
ax[1,0].imshow(Yid[:,:N//2].T.angle(), origin='lower', aspect=T/N)
ax[1,0].set_title(r'arg$Y_{id}$')

   
for m, a in enumerate(ax.flat):
    a.set_yticks(range(0,N//2,N//8), [f'{n/N*fs/1e6:.1f}MHz' for n in range(0,N//2,N//8)])
    a.set_xticks(range(0,T,T//4), [f'{float(f[t])/1e6:.1f}MHz' for t in range(0,T,T//4)])
    a.grid()
    if m//2==1: a.set_xlabel('ideal frequency')
    if m%2==0: a.set_ylabel('FFT frequency')
    

fig.savefig('/home/dgotzens/thesis/figures/interference_test_fft.pdf')

In [40]:
plt.figure(layout='constrained')
plt.plot(Yid.abs().argmax(-1)*fs/N, Y.abs().argmax(-1)*fs/N, label=r'arg max$|Y|$')
plt.plot(Yid.abs().argmax(-1)*fs/N, Yid.abs().argmax(-1)*fs/N, 'k:', label=r'arg max$|Y_{id}|$')
plt.gca().yaxis.set_major_formatter(EngFormatter('Hz'))
plt.gca().xaxis.set_major_formatter(EngFormatter('Hz'))
plt.xlabel('ideal')
plt.ylabel('estimate')
plt.xlim(f0-2e5, f0+2e5)
plt.ylim(f0-2e5, f0+2e5)
plt.grid()
plt.legend()
plt.gcf().set_size_inches(0.5*pdf.a4_textwidth, 0.5*pdf.a4_textwidth)
plt.savefig('/home/dgotzens/thesis/figures/interference_test_peak.pdf')

In [39]:
nid = Yid.abs().argmax(-1)
n = Y.abs().argmax(-1)
print(Yid.shape)
plt.figure(layout='constrained')
plt.plot(nid/N*fs, 180/pi*Yid.angle()[range(T),nid], 'k:', label='ideal phase')
plt.plot(nid/N*fs, 180/pi*Y.angle()[range(T),nid], label='degraded phase at $f_{max,id}$')
plt.plot(nid/N*fs, 180/pi*Y.angle()[range(T),n], label='degraded phase at $f_{max}$')
plt.gca().yaxis.set_major_formatter(EngFormatter('°'))
#plt.xticks(torch.linspace(f0-2e5, f0+2e5, 5))
plt.gca().xaxis.set_major_formatter(EngFormatter('Hz'))
# plt.xlabel('$f_{id}$')
plt.ylabel('phase')
plt.xlabel('ideal frequency')
plt.xlim(omega0/2/pi-2e5, omega0/2/pi+2e5)
# plt.ylim(omega0/2/pi-2e5, omega0/2/pi+2e5)
plt.grid()
plt.legend()
plt.gcf().set_size_inches(0.5*pdf.a4_textwidth, 0.5*pdf.a4_textwidth)
plt.savefig('/home/dgotzens/thesis/figures/interference_peak_phase.pdf')

torch.Size([1000, 65536])
